<a href="https://colab.research.google.com/github/clizar5302/PCC-Risk-Analysis-Prototype/blob/main/RiskAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Functions and Load the File

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
from datetime import datetime

# Prompt the user to upload a file
uploaded = files.upload()

# Extract the file name (it's the key of the uploaded dict)
file_name = next(iter(uploaded))

# Load the .xls file using pandas and xlrd engine
df = pd.read_excel(file_name, engine='xlrd')

# Preview the data to ensure it's loaded correctly
print(df.head())

Saving MTD Loan Data TEST.xls to MTD Loan Data TEST (1).xls
   Days in Month          Loan Name  Advance  Line Amount  Beginning Loan  \
0           24.0  SAMPLE Customer 1     0.80    2500000.0       1750000.0   
1           24.0  SAMPLE Customer 2     0.85    7500000.0       6897800.0   
2           24.0  SAMPLE Customer 3     0.85    5000000.0       3587906.0   
3           24.0  SAMPLE Customer 4     0.85    2500000.0       1369087.0   
4           24.0  SAMPLE Customer 5     0.85    3000000.0       1413131.0   

   Beginning Collateral Last Imported Aging Date  Gross Collateral  \
0          2.437500e+06               2024-07-31      2.690826e+06   
1          8.865059e+06               2024-08-31      9.977064e+06   
2          4.721066e+06               2024-09-30      4.773793e+06   
3          1.860691e+06               2024-09-30      2.117039e+06   
4          1.962507e+06               2024-09-30      2.153506e+06   

      Ineligible Loan Balance  ...  MTD Advances  MTD Di

Define Risk Score Calculation Function

In [ ]:
# Step 4: Define a function to calculate the Extended Risk Score
def calculate_risk_score(row, current_date):
    score = 0

    # 1. Last Imported Aging Date
    aging_date = pd.to_datetime(row['Last Imported Aging Date'], errors='coerce')
    if not pd.isna(aging_date):
        days_aged = (current_date - aging_date).days
        if 45 <= days_aged < 60:
            score += 3
        elif days_aged >= 60:
            score += 5

    # 2. Ineligible as a percentage of Gross Collateral
    if row['Ineligible'] > 0.10 * row['Gross Collateral']:
        score += 1

    # 3. Availability
    if row['Availability'] < 0:
        if row['Availability'] < -0.10 * row['Gross Collateral']:
            score += 5
        elif row['Availability'] < -0.05 * row['Gross Collateral']:
            score += 3
        else:
            score += 2

    # 4. AR Turn in Days
    ar_turn_days = row['AR Turn Days']
    if 45 <= ar_turn_days < 60:
        score += 1
    elif 60 <= ar_turn_days < 90:
        score += 3
    elif ar_turn_days >= 90:
        score += 5

    # 5. AR Turn compared with Historical Average
    ar_turn_diff = row['AR Turn Days'] - row['Historical Average A/R Turn']
    if 5 <= ar_turn_diff < 10:
        score += 3
    elif 10 <= ar_turn_diff < 15:
        score += 5
    elif ar_turn_diff >= 15:
        score += 10

    # 6. Dilution
    dilution = row['Dilution']
    if 5 <= dilution < 10:
        score += 3
    elif 10 <= dilution < 15:
        score += 5
    elif dilution >= 15:
        score += 10

    # 7. Dilution compared with Historical Average
    dilution_diff = row['Dilution'] - row['Historical Average Dilution']
    if 5 <= dilution_diff < 10:
        score += 3
    elif 10 <= dilution_diff < 15:
        score += 5
    elif dilution_diff >= 15:
        score += 10

    # 8. Risk Rating
    risk_rating = row['Risk Rating']
    if risk_rating == 5:
        score += 3
    elif risk_rating == 6:
        score += 5

    return score

Calculate Extended Risk Score

In [ ]:
# Step 5: Calculate the Extended Risk Score for each customer
current_date = datetime.now()
df['Extended Risk Score'] = df.apply(lambda row: calculate_risk_score(row, current_date), axis=1)

# Step 6: Sort the dataframe by the Extended Risk Score in descending order
df_sorted = df.sort_values(by='Extended Risk Score', ascending=False)

# Step 7: Display the top 5 riskiest customers based on the Extended Risk Score
print("\nTop 5 customers based on Extended Risk Score:")
print(df_sorted[['Loan Name', 'Extended Risk Score']].head(5))


Top 5 customers based on Extended Risk Score:
           Loan Name  Extended Risk Score
2  SAMPLE Customer 3                   20
1  SAMPLE Customer 2                   19
5  SAMPLE Customer 6                   10
0  SAMPLE Customer 1                    5
8  SAMPLE Customer 9                    5


Display Top 5 Customers by Other Factors

In [ ]:
# Convert the 'Loan Balance' column to numeric, coercing errors to NaN
df_sorted['Loan Balance'] = pd.to_numeric(df_sorted['Loan Balance'], errors='coerce')

# Optionally, drop rows with NaN values in 'Loan Balance'
df_sorted = df_sorted.dropna(subset=['Loan Balance'])

# Sort the DataFrame by different columns and get the top 5 customers for each category
top_5_size_of_loan = df_sorted.sort_values(by='Loan Balance', ascending=False).head(5)
top_5_ar_turn_days = df_sorted.sort_values(by='AR Turn Days', ascending=False).head(5)
top_5_ar_turn_vs_historical = df_sorted.sort_values(by='Historical Average A/R Turn', ascending=False).head(5)
top_5_dilution = df_sorted.sort_values(by='Dilution', ascending=False).head(5)
top_5_dilution_vs_historical = df_sorted.sort_values(by='Historical Average Dilution', ascending=False).head(5)

# Print the top 5 results for each category

print("\nTop 5 customers by Loan Balance (Size of Loan):")
print(top_5_size_of_loan[['Loan Name', 'Loan Balance']])

print("\nTop 5 customers by AR Turn Days:")
print(top_5_ar_turn_days[['Loan Name', 'AR Turn Days']])

print("\nTop 5 customers by AR Turn compared with Historical Average:")
print(top_5_ar_turn_vs_historical[['Loan Name', 'Historical Average A/R Turn']])

print("\nTop 5 customers by Dilution:")
print(top_5_dilution[['Loan Name', 'Dilution']])

print("\nTop 5 customers by Dilution compared with Historical Average:")
print(top_5_dilution_vs_historical[['Loan Name', 'Historical Average Dilution']])


Top 5 customers by Loan Balance (Size of Loan):
            Loan Name  Loan Balance
8   SAMPLE Customer 9   33163537.65
5   SAMPLE Customer 6   23955356.90
6   SAMPLE Customer 7   17716788.75
1   SAMPLE Customer 2    7580832.65
9  SAMPLE Customer 10    4722656.60

Top 5 customers by AR Turn Days:
           Loan Name  AR Turn Days
2  SAMPLE Customer 3    115.446420
1  SAMPLE Customer 2     84.754022
4  SAMPLE Customer 5     51.495405
6  SAMPLE Customer 7     47.777350
8  SAMPLE Customer 9     44.332798

Top 5 customers by AR Turn compared with Historical Average:
           Loan Name  Historical Average A/R Turn
2  SAMPLE Customer 3                         91.0
1  SAMPLE Customer 2                         60.0
4  SAMPLE Customer 5                         52.0
0  SAMPLE Customer 1                         45.0
8  SAMPLE Customer 9                         45.0

Top 5 customers by Dilution:
           Loan Name  Dilution
5  SAMPLE Customer 6  0.231752
0  SAMPLE Customer 1  0.091374
6  SAM

Normalize Scores

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Normalize the Extended Risk Score
df['Normalized Risk Score'] = sigmoid(df['Extended Risk Score'])

# Preview of Normalized Risk Scores
print("\nPreview of Normalized Risk Scores:")
print(df[['Loan Name', 'Extended Risk Score', 'Normalized Risk Score']].head(5))


Preview of Normalized Risk Scores:
           Loan Name  Extended Risk Score  Normalized Risk Score
0  SAMPLE Customer 1                    5               0.993307
1  SAMPLE Customer 2                   19               1.000000
2  SAMPLE Customer 3                   20               1.000000
3  SAMPLE Customer 4                    0               0.500000
4  SAMPLE Customer 5                    1               0.731059
